In [17]:
import pandas as pd
import numpy as np

import time
import copy

from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# from sklearn.cluster import Birch
# import matplotlib.pyplot as plt




In [18]:
def k_means_model_with_best_sil_score(X, random_seed = 0, kmax = 10):
  # sil = []

  # for k in range(2, kmax+1):
  #   kmeans = KMeans(n_clusters = k, random_state=random_seed).fit(X)
  #   sil.append(silhouette_score(X, kmeans.labels_, metric = 'euclidean'))

  # print(sil)
  # return sil.index(max(sil))+2

  highest_score = 0
  k_means_model = None

  for ks in range(2, kmax+1):
    kmeans = KMeans(n_clusters = ks, random_state=random_seed).fit(X)
    cur_score = silhouette_score(X, kmeans.labels_, metric = 'euclidean')
    if cur_score > highest_score:
      highest_score = cur_score
      k_means_model = kmeans

  return k_means_model

In [19]:
# data structure
class node:
    def __init__(self, entity, centre, index = None, up_lv = None, down_lv = None):
        self.entity = entity
        self.centre = centre # cluster centre (vector)
        self.index = index # cashe of search result, first 100? result close to centre (list of index)
        self.up_lv = up_lv # super-cluster (node)
        self.down_lv = down_lv # sub-cluster (lsit of node)

    def info(self):
        print(f"entity:{self.entity}\ndown_lv: {len(self.down_lv) if self.down_lv else 0}\nno. of node: {len(self.index)}\ncentre:{self.centre}",end="\n")
        

# graph tree (cluster)
class graph_tree:
    def __init__(self, attr_name, X, min_cluster_size, searhing_query = None):
        self.attr_name = attr_name
        self.X = X
        self.min_cluster_size = min_cluster_size
        self.searhing_query = searhing_query

        # create root node
        init_centre = np.mean(X, axis=0)
        init_idx = np.array([i for i in range(len(X))])
        self.root = node(entity=sorted(list(zip(self.attr_name, init_centre))), centre=init_centre, index=init_idx[np.argsort(np.sum(abs(X-init_centre), axis=1))])
        del (init_centre, init_idx)
        print("root node is created")

    def sim(self, vector_1, vector_2):
        return sum((vector_1-vector_2)**2)

    def search(self, query_vector):
        self.searhing_query = query_vector
        cur = self.root
        
        while cur.down_lv:
            cur_centre_distance = self.sim(query_vector, cur.centre)
            # print(cur_centre_distance)

            closest = cur_centre_distance
            closest_node_position = -1

            for node in range(len(cur.down_lv)):
                if self.sim(query_vector, cur.down_lv[node].centre) < closest:
                    closest = self.sim(query_vector, cur.down_lv[node].centre)
                    closest_node_position = node

            if closest_node_position < 0:
                break
            
            cur = cur.down_lv[closest_node_position]

        return cur
        


    def build_tree(self):
        queue = []
        queue.append(self.root)
        
        lv = 0
    
        while queue:
            size = len(queue)
            print("queue", size, queue)
            print(f"\n---------------------------------- level {lv} no of node {size} ----------------------------------------")
            for node_in_this_lv in range(size):
                print(f"\nhandling node {node_in_this_lv}, info:")
                # k means required parameter and train the model
                cur = queue.pop(0) # get cur node
                print(f"cur node {cur}")
                print(f"no. of cur node data {len(cur.index)}")
                if len(cur.index) > self.min_cluster_size:
                    cur_X = self.X[cur.index] # get cur data by index
                    k_means_model = k_means_model_with_best_sil_score(cur_X) # get kmeans model
                    print(f"optimal K {k_means_model.n_clusters}\n")

                    # # relationship
                    # attr_no = 3
                    # largest_indices = np.argpartition(np.std(k_means_model.cluster_centers_, axis=0), -attr_no)[-attr_no:]
                    # print(largest_indices)

                    # relationship = {}
                    # np.argpartition(np.std(k_means_model.cluster_centers_, axis=0), -attr_no)[-attr_no:]
                    # for i in largest_indices:
                    #     tmp_entity[self.attr_name[i]]=k_means_model.cluster_centers_[groups][i]

                    # create required node
                    new_down_lv = [node(entity=sorted(list(zip(self.attr_name, k_means_model.cluster_centers_[i])), key=lambda item: item[1], reverse=True), centre=k_means_model.cluster_centers_[i], up_lv=cur) for i in range(k_means_model.n_clusters)]
                    distance_to_self_centre = []
                    for i in range(len(cur_X)):
                        distance_to_self_centre.append(sum(abs(cur_X[i]-k_means_model.cluster_centers_[k_means_model.labels_[i]])))

                    sorted_indices = np.argsort(np.array(distance_to_self_centre))
                    sorted_index = cur.index[sorted_indices]
                    sorted_labels = k_means_model.labels_[sorted_indices]
                    for i in range(k_means_model.n_clusters):
                        new_down_lv[i].index = sorted_index[np.where(sorted_labels == i)[0]]

                    cur.down_lv = new_down_lv
                    
                    queue += new_down_lv
                    # print(f"down lv of this node {new_down_lv}")
                else:
                    print("not pass min_cluster_size")
            lv += 1

        return 0
    
    def print_tree(self, simple=1):
        queue = [self.root]
        lv = 0
        while queue:
            print(f"lv: {lv}, node num.: {len(queue)}")
            new_queue = []
            for n in queue:
                if simple == 0:
                    n.info()
                if n.down_lv:
                    new_queue += n.down_lv
            queue = new_queue
            lv += 1

    # def flip(self):
        



In [20]:
raw = pd.read_csv(r'Data\train.csv')
# print(raw)

random_seed = 42

# print(list(raw.columns)[:])

In [31]:
data = raw.iloc[:,2:-1]
data = data.fillna(0)
data = data[:5000]
print(data.shape)
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(data)

# label
y = raw.iloc[:,-1]
# print(y)

col_name = list(raw.columns)[2:-1]
print(col_name)
# print(normalized_data)

(5000, 14)
['Popularity', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_in min/ms', 'time_signature']


In [22]:
test_tree = graph_tree(attr_name = col_name, X = normalized_data, min_cluster_size=200)
# test_tree.show_root()
test_tree.build_tree()

root node is created
queue 1 [<__main__.node object at 0x000001FA02054510>]

---------------------------------- level 0 no of node 1 ----------------------------------------

handling node 0, info:
cur node <__main__.node object at 0x000001FA02054510>
no. of cur node data 5000
optimal K 2

queue 2 [<__main__.node object at 0x000001FA50E09B50>, <__main__.node object at 0x000001FA001FD310>]

---------------------------------- level 1 no of node 2 ----------------------------------------

handling node 0, info:
cur node <__main__.node object at 0x000001FA50E09B50>
no. of cur node data 3195
optimal K 2


handling node 1, info:
cur node <__main__.node object at 0x000001FA001FD310>
no. of cur node data 1805
optimal K 2

queue 4 [<__main__.node object at 0x000001FA00640650>, <__main__.node object at 0x000001FA00457790>, <__main__.node object at 0x000001FA50DAB290>, <__main__.node object at 0x000001FA50DC0C50>]

---------------------------------- level 2 no of node 4 --------------------------

0

In [23]:
test_tree.print_tree(simple=1)

lv: 0, node num.: 1
lv: 1, node num.: 2
lv: 2, node num.: 4
lv: 3, node num.: 10
lv: 4, node num.: 14
lv: 5, node num.: 28
lv: 6, node num.: 10
lv: 7, node num.: 2


In [28]:
seaching = np.mean(normalized_data[100:101], axis=0)
print(seaching)
cur = test_tree.search(seaching)

[0.24242424 0.46751979 0.5035126  0.36363636 0.60243818 0.
 0.03728814 0.62751004 0.79718876 0.21300082 0.12417892 0.69124913
 0.18320677 0.75      ]


In [25]:
def super_cluster(cur):
    # super cluster
    print(f"super cluster")
    if cur.up_lv:
        cur.up_lv.info()
    else:
        print(f"there is no super cluster")
    print("\n", end="")

def neibour_cluster(cur):
    # neibour in the same lv
    print(f"neibour in the same lv")
    if cur.up_lv:
        if len(cur.up_lv.down_lv) > 1:
            for neibour in cur.up_lv.down_lv:
                if neibour != cur:
                    neibour.info()
        else:
            print(f"there is no neibour")
    else:
        print(f"there is no neibour")
    print("\n", end="")

def sub_cluster(cur):
    # sub cluster
    print(f"sub cluster")
    if cur.down_lv:
        for sub in cur.down_lv:
            sub.info()
    else:
        print(f"there is no sub cluster")
    print("\n", end="")


def suround_cluster(cur):
    # cur cluster
    print(f"current cluster")
    cur.info()
    print("\n", end="")
    
    super_cluster(cur)
    neibour_cluster(cur)
    sub_cluster(cur)
    

In [29]:
# specific searching
# cur = cur.down_lv[0]

#generic searching
# cur = cur.up_lv

#simular searching
# cur = cur.up_lv.down_lv[1]

suround_cluster(cur)

current cluster
entity:[('instrumentalness', 0.8157589033821235), ('acousticness', 0.7422068439045438), ('time_signature', 0.6859504132231404), ('loudness', 0.5415391052364475), ('key', 0.5274229902329076), ('tempo', 0.4247294561271386), ('danceability', 0.419428881792459), ('Popularity', 0.3625511311461725), ('energy', 0.2994371221306348), ('valence', 0.21184547152073527), ('liveness', 0.13389375806680498), ('duration_in min/ms', 0.12098703972205746), ('speechiness', 0.033999521031679684), ('mode', 0.0)]
down_lv: 0
no. of node: 121
centre:[0.36255113 0.41942888 0.29943712 0.52742299 0.54153911 0.
 0.03399952 0.74220684 0.8157589  0.13389376 0.21184547 0.42472946
 0.12098704 0.68595041]

super cluster
entity:[('time_signature', 0.705607476635514), ('acousticness', 0.6802179512442292), ('loudness', 0.6493975695312526), ('key', 0.5044604927782498), ('danceability', 0.48923804562320666), ('tempo', 0.43908318824827175), ('energy', 0.3949135636830238), ('Popularity', 0.3762154252808459), ('

In [ ]:
print(normalized_data.shape)
print(np.mean(normalized_data[:100], axis=0))

print(test_tree.root.centre)

(5000, 14)
[0.42919192 0.53548422 0.6287219  0.45818182 0.74335117 0.72
 0.05481575 0.24029304 0.13223035 0.16829518 0.49376186 0.47735809
 0.13475585 0.72      ]
[0.43906465 0.52589225 0.66144496 0.47741818 0.7539839  0.639
 0.06208811 0.25001819 0.13598254 0.1839972  0.47853415 0.48415781
 0.14202648 0.7291    ]


In [ ]:
from collections import Counter

"""
create a new index (something like tf-idf)
data reduction rate
vs
accurate
"""
def score(search_data, acc, smoothing = 0.001, lam = 0.5):
    return lam*(acc/100)+(1-lam)*(1-(search_data/(1000+smoothing)))

my_score = []
normal_kmeans_score = []

tmp = [i*100 for i in range(10)]
for i in tmp:
    b = i
    u = i+100
    # print(normalized_data[b:u])
    print(f"-- mine result{i}")
    seaching = np.mean(normalized_data[b:u], axis=0)
    result = test_tree.search(seaching)
    
    print("search number: ", len(result.index))
    right = np.where((result.index>=b) & (result.index<u))[0] # return index
    print("hitted right result:", len(right))
    my_score.append(score(len(result.index), len(right)))

    ############
    print("-- kmeans result")
    checking = KMeans(n_clusters=10, random_state=0).fit(normalized_data)

    counts1 = Counter(checking.labels_)
    print(counts1)

    counts2 = Counter(checking.labels_[b:u])
    print(counts2)
    belongs_group = counts2.most_common()[0][0]
    print("belongs to group: ", belongs_group, "\n")
    # print(counts1.get(belongs_group), counts2.get(belongs_group))
    normal_kmeans_score.append(score(counts1.get(belongs_group), counts2.get(belongs_group)))

    

-- mine result0
search number:  5000
hitted right result: 100
-- kmeans result
Counter({9: 944, 0: 659, 3: 656, 1: 619, 4: 607, 7: 577, 5: 299, 8: 281, 6: 191, 2: 167})
Counter({9: 22, 1: 17, 0: 13, 4: 12, 7: 10, 3: 9, 8: 6, 5: 5, 2: 5, 6: 1})
belongs to group:  9 

-- mine result100
search number:  5000
hitted right result: 100
-- kmeans result
Counter({9: 944, 0: 659, 3: 656, 1: 619, 4: 607, 7: 577, 5: 299, 8: 281, 6: 191, 2: 167})
Counter({3: 17, 9: 15, 4: 15, 7: 11, 5: 10, 1: 10, 0: 9, 6: 6, 8: 5, 2: 2})
belongs to group:  3 

-- mine result200
search number:  5000
hitted right result: 100
-- kmeans result
Counter({9: 944, 0: 659, 3: 656, 1: 619, 4: 607, 7: 577, 5: 299, 8: 281, 6: 191, 2: 167})
Counter({9: 21, 1: 14, 3: 12, 0: 12, 7: 8, 8: 7, 4: 7, 2: 7, 5: 6, 6: 6})
belongs to group:  9 

-- mine result300
search number:  5000
hitted right result: 100
-- kmeans result
Counter({9: 944, 0: 659, 3: 656, 1: 619, 4: 607, 7: 577, 5: 299, 8: 281, 6: 191, 2: 167})
Counter({9: 21, 0: 17, 4

In [ ]:
print(my_score)
print(normal_kmeans_score)

print(sum(my_score))
print(sum(normal_kmeans_score))

[-1.4999975000024999, -1.4999975000024999, -1.4999975000024999, -1.4999975000024999, -1.4999975000024999, -1.4999975000024999, -1.4999975000024999, -1.4999975000024999, -1.4999975000024999, -1.4999975000024999]
[0.13800047199952797, 0.25700032799967204, 0.13300047199952797, 0.13300047199952797, 0.27550032949967046, 0.12300047199952799, 0.2965002884997115, 0.28650030349969646, 0.12300047199952799, 0.14300047199952798]
-14.999975000025001
1.908504081495918
